In [5]:
#aiko package
#Denis Titov
#16.01.2019

"""
aiko gives read/write functionality for a .hdf5 database.
so far it only works on that specific database :/
*path*/snapdir_000/snapshot_000.0.hdf5
"""

import h5py
import numpy as np
import sys
import os
import pandas as pd
import re

In [86]:
#class for reading/writing/handlin of snapshot directories, managed by aiko
class snapshot():
    #sets path of snapshot dir (corrects for missing slash at end of path)
    def __init__(self, snapPath, blockNames, particleTypes):
        if snapPath[-1] is not "/":
            snapPath = snapPath + "/"
        self.snapPath = snapPath
        self.blockNames = blockNames
        self.particleTypes = particleTypes
        self.particleTypeIndexes = self.get_particleType_index()
        self.numSnapdirs = self.get_numSnapdirs()
        self.currentSnapshot = self.get_snapshot()

    #searches for a file with correct name, opens and gets number of snaps from header
    def get_numSnapdirs(self):
        for i in os.listdir(self.snapPath):
            if re.search("\.\d\.", i):
                fileName = i
                break
        filePath = self.snapPath + fileName
        file = h5py.File(filePath, 'r')
        attrs = file['/Header'].attrs
        numFilesPerSnapshot = attrs["NumFilesPerSnapshot"]
        return numFilesPerSnapshot

    #converts words of particleTypes to particleTypeIndexes
    def get_particleType_index(self):
        indexList = []
        index = {"Gas":0, "DM":1, "Darkmatter":1, "Stars":4}
        for particleType in self.particleTypes:
            converted = index[particleType]
            indexList.append(converted)
        return indexList

    #returns current snapshot
    def get_snapshot(self):
        for i in os.listdir(self.snapPath):
            if re.search("\_\d{3}\.", i):
                snapshot = re.findall("\_\d{3}\.", i)[0][1:4]
                return int(snapshot)

    #returns list of all filenames in dir
    def get_snapdirList(self):
        return os.listdir(self.snapPath)

    #returns
    def read_arepo(self):
        dfSnapList = {}
        snapIter = 0
        for filename in self.get_snapdirList():
            file = h5py.File(self.snapPath +  filename, "r")
            dfFileList = {}
            for particleTypeIndex in self.particleTypeIndexes:
                dfStructure = {}
                fileIter = 0
                for blockName in self.blockNames:
                    h5Path = "/PartType{}/{}".format(particleTypeIndex, blockName)
                    print(h5Path)
                    ID  = np.uint32(file[h5Path].value)
                    dfStructure["ID"] = ID
                    if blockName == "Coordinates":
                        coords = np.float32(file[h5Path].value)
                        dfStructure["posX"] = coords[:,0]
                        dfStructure["posY"] = coords[:,1]
                        dfStructure["posZ"] = coords[:,2]
                    if blockName == "Velocities":
                        vels = np.float32(file[h5Path].value)
                        dfStructure["velX"] = vels[:,0]
                        dfStructure["velY"] = vels[:,1]
                        dfStructure["velZ"] = vels[:,2]
                    #if blockName == "Density" and particleTypeIndex == 0:
                     #   print(particleTypeIndex, blockName, filename)
                     #   dens = np.float32(file[h5Path].value)
                     #   dfStructure["Dens"] = dens
                dfStructure["Snap"] = ID*0+self.currentSnapshot
                dfStructure["Type"] = ID*0+particleTypeIndex
                dfCurrent = pd.DataFrame(dfStructure)
                dfFileList[fileIter] = dfCurrent
                fileIter += 1
            dfFile = pd.concat(dfCurrent)
            dfSnapList[snapIter] = dfFile
            snapIter =+ 1
        dfSnap = pd.concat(dfSnapList)
        return dfSnap

#{"x":pos[:,0],"y":pos[:,1],"z":pos[:,2],"dens":dens , "ID":ID, "snap":ID*0+s}
#pos = np.float32(f['/PartType0/Coordinates'].value)
a = snapshot(snapPath="/store/clues/HESTIA/RE_SIMS/2048/GAL_FOR/37_11/output/snapdir_127", particleTypes=["Gas", "Stars"], blockNames=["Coordinates", "Velocities", "Density"])

print(a.read_arepo())

/PartType0/Coordinates
/PartType0/Velocities
/PartType0/Density


/PartType4/Coordinates
/PartType4/Velocities
/PartType4/Density


KeyError: "Unable to open object (object 'Density' doesn't exist)"

In [0]:
class aiko():
    def __init__(self, path):
        self.path = path
    
    def read_repo():
        pathList = aiko.get_path_list()
        pathList = pathList[-48:]
        for i in range(len(pathList)):
            #TODO read funktion inform
            f = h5py.File(pathList[i], "r")
            s = int(pathList[i][62:65])
            #type=0
            pos = np.float32(f['/PartType0/Coordinates'].value)
            ID  = np.uint32(f['/PartType0/ParticleIDs'].value)
            dens = np.array(f['PartType0/Density'])
            df0=pd.DataFrame({"x":pos[:,0],"y":pos[:,1],"z":pos[:,2],"dens":dens , "ID":ID,         "snap":ID*0+s})
            dfallList[i]=df0
            print(i)
            #TODO close f
        dfa = pd.concat(dfallList)
        print("Done!")

    def get_path_list():
        pathList = []
        for i in range(aiko.numSnapdirs):
            currentSnapdir = "snapdir_" + "%03d" % i + "/"
            for j in range(aiko.numSnapshots):
                currentSnapshot = "snapshot_" + "%03d" % i + "." + str(j) + ".hdf5"
                currentPath = aiko.path + currentSnapdir + currentSnapshot
                pathList.append(currentPath)
        return pathList

    

"""
TODO:

read FILE FOLDER REPOSITORY ETC.

write TO DATABASE, MULTIPLE FILES ETC.

convert TO PANDAS, TO NUMPY (SELECTION, SLICE) ETC.

give direkt dirname
"""